In [8]:
# 데이터를 가져오는 부분
from keras.datasets import boston_housing
(train_data, train_labels),(test_data, test_labels) = boston_housing.load_data()

In [9]:
# train / test 데이터 모양 확인
print(train_data.shape)
print(test_data.shape)

(404, 13)
(102, 13)


In [10]:
# 전체 데이터 보기
print(train_data[0])

[  1.23247   0.        8.14      0.        0.538     6.142    91.7
   3.9769    4.      307.       21.      396.9      18.72   ]


In [11]:
# 데이터 전처리 (Scaling)

mean = train_data.mean(axis=0)
train_data -= mean   #train_data = train_data - mean 과 같은 표현
std = train_data.std(axis=0)
train_data /= std

train_data[0]

array([-0.27224633, -0.48361547, -0.43576161, -0.25683275, -0.1652266 ,
       -0.1764426 ,  0.81306188,  0.1166983 , -0.62624905, -0.59517003,
        1.14850044,  0.44807713,  0.8252202 ])

In [12]:
# 모델의 구성
from keras import models
from keras import layers

def build_model():
  model = models.Sequential()

  # 입력 되는 첫 번째 레이어 데이터 내용
  model.add(layers.Dense(64, activation='relu', input_shape=(train_data.shape[1],)))

  # 입력 되는 두 번째 레이어 데이터 내용
  model.add(layers.Dense(64, activation='relu'))

  model.add(layers.Dense(1))

  # 모델의 컴파일
  model.compile(optimizer='rmsprop',
                loss='mse',
                metrics=['mse'])
  return model

In [13]:
# K-folder 검증을 사용한 훈련 검증 (K-folder 검증이랑 몇 조각으로 나눈 개념)

import numpy as np

k = 4                                       # folder 개수
num_val_samples = len(train_data) // k      # 한 folder의 데이터 수
all_scores = []

# k 의 숫자 만큼 반복
for i in range(k):
  print('처리중인 폴드 #', i)

  # 검증 데이터의 준비
  # 시작 점                                   # 끝 점
  val_data = train_data[i * num_val_samples: (i + 1) * num_val_samples]
  # 라벨 쪼개기
  val_labels = train_labels[i * num_val_samples: (i + 1) * num_val_samples]

  # 학습 데이터의 준비
  data1 = train_data[:i * num_val_samples]
  data2 = train_data[(i+1) * num_val_samples :]
  data1_labels = train_labels[:i * num_val_samples]
  data2_labels = train_labels[(i+1) * num_val_samples :]

  partial_train_data = np.concatenate([data1, data2], axis=0)
  partial_train_labels = np.concatenate([data1_labels, data2_labels], axis=0)

  # 모델의 학습
  model = build_model()
  model.summary()

  model.fit(partial_train_data, partial_train_labels, epochs=500, batch_size=128, verbose=0)

  # 모델의 검증
  val_mse, val_mae = model.evaluate(val_data, val_labels)
  print(val_mse, val_mae)
  all_scores.append(val_mae)


처리중인 폴드 # 0
Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_12 (Dense)            (None, 64)                896       
                                                                 
 dense_13 (Dense)            (None, 64)                4160      
                                                                 
 dense_14 (Dense)            (None, 1)                 65        
                                                                 
Total params: 5,121
Trainable params: 5,121
Non-trainable params: 0
_________________________________________________________________
4/4 [==============================] - 0s 4ms/step - loss: 7.5791 - mse: 7.5791
7.579145908355713 7.579145908355713
처리중인 폴드 # 1
Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_15 (Dense)     

In [14]:
np.mean(all_scores)

10.532718300819397